# 트리의 앙상블

## 랜덤포레스트

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


## 히스토그램 기반 부스팅

In [10]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [11]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [14]:
# 이전 셀에서 사이킷런 버전을 재설치했다면 세션을 다시 시작하고
# 다음 코드를 다시 실행하세요.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate


wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [15]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
xgb._estimator_type = "classifier"
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

ModuleNotFoundError: No module named 'xgboost'

#### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


- 앙상블 학습
    - 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘
- 랜덤 포레스트
    - 대표적인 결정 트리 기반의 앙상블 학습 방법
    - 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만드는 것이 특징
- 엑스트라 트리
    - 랜덤 포레스트와 비슷함
    - 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않음
    - 대신 랜덤하게 노드를 분할해 과대적합을 감소시킴
- 그레이디언트 부스팅
    - 랜덤 포레스트나 엑스트라 트리와 달리 결정 트리를 연속적으로 추가하여 손실 함수를 최소하는 앙상블 방법
    - 이런 이유로 훈련 속도가 조금 느리지만 더 좋은 성능을 기대 할 수 있음.
- 히스토그램 기반 그레이디언트 부스팅
    - 그레이디언트 부스팅의 속도를 개선한 것
    - 안정적인 결과와 높은 성능으로 매우 인기가 높음

### 핵심 패키지와 함수
- scikit-learn
    - RandomForestClassifier
        - 랜덤 포레스트 분류 클래스
        - n_estimators 매개변수
            - 앙상블을 구성할 트리의 개수 지정
            - 기본값은 100
        - criterion 매개변수
            - 불순도를 지정하며 기본값은 지니 불순도를 의미하는 'gini'
            - 엔트로피 불순도는 'entropy'
        - max_depth
            - 트리가 성장할 최대 깊이를 지정
            - 기본값은 None
            - 지정하면 리프노드가 순수하거나 min_samples_split 보다 샘플 수가 적을 때까지 성장함
        - min_samples_split
            - 노드를 나누기 위한 최소 샘플 개수
            - 기본값은 2
        - max_features 매개변수
            - 최적의 분할을 위해 탐색할 특성의 개수를 지정
            - 기본값은 sqrt로 특성 개수의 제곱근
        - bootstrp 매개변수
            - 부트스트랩 샘플을 사용할지 지정.
            - 기본값은 True
        - oob_score
            - OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정
            - 기본값은 False
        - n_jobs 매개변수
            - 병렬 실행에 사용할 CPU 코어 수를 지정
            - 기본값은 1
                - 하나의 코어 사용
            - -1로 지정하면 시스템에 있는 모든 코어를 사용
    - ExtraTreesClassifier
        - 엑스트라 트리 분류 클래스
        - n_extimators, criterion, max_depth, min_samples_split, max_features 매개변수
            - 랜덤 포레스트와 동일
        - bootstrap 매개변수
            - 부트스트랩 샘플을 사용할지 지정
            - 기본값은 False
        - oob_score
            - OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정
            - 기본값은 False
        - n_jobs 매개변수
            - 병렬 실행에 사용할 CPU 코어 수를 지정
            - 기본값은 1
                - 하나의 코어 사용
            - -1로 지정하면 시스템에 있는 모든 코어를 사용
    - GradientBoostingClassifier
        - 그레이디언트 부스팅 분류 클래스
        - loss 매개변수
            - 손실 함수 지정
            - 기본값은 로지스틱 손실 함수를 의미하는 'log_loss'
        - learning_rate 매개변수
            - 트리가 앙상블에 기여하는 정도를 조절
            - 기본값은 0.1
        - n_estimators 매개변수
            - 부스팅 단계를 수행하는 트리의 개수
            - 기본값은 100
        - subsample 매개변수
            - 사용할 훈련 세트의 샘플 비율을 지정
            - 기본값은 0.1
        - max_depth 매개변수
            - 개별 회귀 트리의 최대 깊이
            - 기본값은 3
    - HistGradientBoostingClassifier 
        - 히스토그램 기반 그레이디언트 부스팅 분류 클래스
        - learning_rate 매개변수
            - 학습룰 또는 감쇠율이라고 함.
            - 기본값은 0.1
            - 1.0이면 감쇠가 전혀 없음
        - max_iter
            - 부스팅 단계를 수행하는 트리의 개수
            - 기본값은 100
        - max_bins
            - 입력 데이터를 나눌 구간의 개수
            - 기본값은 255
                - 이보다 크게 지정 할 수 없음
            - 여기에 1개의 구간이 누락된 값을 위해 추가됨 

### 확인 문제
1. 여러 개의 모델을 훈련시키고 각 모델의 예측을 취합하여 최종 결과를 만드는 학습 방식을 무엇이라고 부르나요?
    1. 단체 학습
    2. 오케스트라 학습
    3. 심포니 학습
    4. 앙상블 학습
    > 정답: 4

2. 다음 중 비정형 데이터에 속하는 것은 무엇인가요?
    1. 엑셀 데이터
    2. CSV 데이터
    3. 데이터베이스 데이터
    4. 이미지 데이터
    > 정답: 4

3. 다음 알고리즘 중 기본적으로 부트스트랩 샘플을 사용하는 알고리즘은 무엇인가요?
    1. 랜덤 포레스트
    2. 엑스트라 트리
    3. 그레이디언트 부스팅
    4. 히스토그램 기반 그레이디언트 부스팅
    > 정답: 1

4. 다음 중 순서대로 트리를 추가하여 앙상블 모델을 만드는 방법은 무엇인가요?
    1. 결정 트리
    2. 랜덤 포레스트
    3. 엑스트라 트리
    4. 그레이디언트 부스팅
    > 정답: 4